In [13]:
import itertools
import datetime
import random
import math


class Env:
    def __init__(self):
        philadelphia = City('Philadelphia',800000, temp=70, unemployment_rate=.1, gas=3.5)
        nyc = City('New York City', 10000000, temp=67, unemployment_rate=.05, gas=5)
        washington = City("Washington DC", 800000, temp=72, unemployment_rate=.07, gas=4)
        self.cities = [philadelphia, nyc, washington]
        
        # Using list comprehension and Trip class to generate the trips
        self.trips = [Trip(origin, destination) for origin, destination in itertools.combinations(self.cities, 2)]
        
        self.date = datetime.datetime.now()

    def add_city(self, city):
        self.cities.append(city)

    def add_trip(self, origin_city, destination_city):
        self.trips.append(Trip(origin_city, destination_city))
        
    def step(self):
        # update all the values in each city, simulating change that happens over time
        for city in self.cities:    
            city.update_gas_prices()
            city.update_temperature()

    def score(self):
        total_demand = 0
        for trip in self.trips:
            sum += trip.calculate_demand()
        return math.sqrt(sum)
        # given the changes to gas_prices and 
    def actions(self):
        for trip in self.trips:
            trip.average_cost *= random.choice(.99,1, 1.01)
            
            

class City:
    def __init__(self, name, pop, temp, unemployment_rate, gas):
        self.name = name
        self.population = pop
        self.base_temperature = temp
        self.temperature = temp
        self.ue_rate = unemployment_rate
        self.base_gas_prices = gas
        self.gas_prices = gas

    def update_gas_prices(self):
        outcome = random.choice([.99, 0,1.01])
        if outcome == .99 or outcome == 1.01:
            self.gas_prices = self.base_gas_prices * outcome
    
        # there is 1% chance that this gas price will become the new 'base'
        if random.randint(0,100) == 52:
            self.base_gas_prices = self.gas_prices
    
    def update_temperature(self):
        # adding variability to daily temperature
        self.temperature = random.Random.uniform(self.base_temperature-5, self.base_temperature+5)
        
        # there is 1% chance that this temperature will become the new 'base'
        if random.randint(0,100) == 36:
            self.base_temperature = self.temperature 
                        
    ### for now we are keeping unemployment rate + population fixed
        

class Trip:
    def __init__(self, origin:City, destination:City, departure_time=None):
        self.origin = origin
        self.destination = destination
        self.departure_time = departure_time if departure_time else datetime.datetime.now()
        self.average_cost = 100
        
        
    # more demand during business hours
    def calc_hour_modifier(self):
        # weekdays
        if self.departure_time.day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]:
            if self.departure_time.hour in range(8,17):
                return 0
            return -50
            
        # weekends
        if self.departure_time.hour in range(10,20):
            return 0
        return -50
    
    def calculate_weather_modifier(self):
        # every degree above or below 68 degrees impacts demand by .5 %
        return -1 * abs(1/2 * (self.origin.temperature + self.destination.temperature) - 68)
    
    def calculate_price_modifier(self):
        return (self.average_cost - 100) ** 3
    
    def calculate_demand(self):
        # This is a simple algorithm. We're gonna assume that the base demand for a trip is 1 % of origin * 1% destination pop
        base_demand = self.origin.population * 0.01 * self.destination.population * 0.01
        
        
        '''modifiers'''
        # more expensive gas, more people take train
        gas_modifier = (self.origin.gas_prices - 3) * 2 + self.destination.gas_prices - 3 
        
        # more unemployment, less rides
        unemployment_modifier = (self.origin.ue_rate + self.destination.ue_rate)/2 
        
        # pleasant weather = more people, extreme weather = less
        weather_modifier = self.calculate_weather_modifier()
        
        #late / early times are punished
        datetime_modifier = self.calc_hour_modifier()
        
        return base_demand * .01 * (gas_modifier + unemployment_modifier + datetime_modifier + weather_modifier + price_modifier)
    
    def __str__(self):
        return f"Trip from {self.origin.name} to {self.destination.name} at {self.departure_time}"



In [14]:
env = Env()

for i in range(100):
    env.step()
    env.actions()
    print(f"The total score for this round is: {env.score()}")
    

    
    

AttributeError: 'Env' object has no attribute 'next_step'